In [31]:
### import sys
import os
import numpy as np
import pandas as pd

sys.path.append(os.path.abspath('../..'))

from offshore_wind_nj.config import INTERIM_DATA_DIR, ERA5_DATA_DIR, BUOY_DATA_DIR
import matplotlib.pyplot as plt
from pyproj import Proj, Transformer

In [2]:
os.listdir(BUOY_DATA_DIR)

['44025h2023.txt', '44025h2023.txt.gz', '44065h2023.txt', '44065h2023.txt.gz']

# Buoys path

In [3]:
buoy_44025_path = os.path.join(BUOY_DATA_DIR, '44025h2023.txt')
buoy_44065_path = os.path.join(BUOY_DATA_DIR, '44065h2023.txt')

## Function to read Buoy files

In [11]:
def read_buoy_data(file_path):
    """
    Reads a buoy data file, extracts headers and units, and returns a DataFrame 
    with formatted column names.
    
    Parameters:
        file_path (str): Path to the buoy data file.
    
    Returns:
        pd.DataFrame: Processed buoy data with column names including units.
    """
    # Read header and units
    header = pd.read_csv(file_path, sep=r"\s+", nrows=1)
    units = pd.read_csv(file_path, sep=r"\s+", skiprows=0, nrows=1)
    
    # Create new column names with units
    new_columns = [f"{col} ({unit})" for col, unit in zip(header.columns, units.iloc[0])]
    
    # Read the main data with updated column names
    data = pd.read_csv(file_path, sep=r"\s+", skiprows=2, names=new_columns)
    
    return data

# Load the datasets

In [16]:
buoy_44025 = read_buoy_data(buoy_44025_path)
buoy_44065 = read_buoy_data(buoy_44065_path)

# Data matching

In [30]:
'2023-06-16T11:00:00'
buoy_44065[(buoy_44065['MM (mo)']==6) & (buoy_44065['DD (dy)']==16) & (buoy_44065['hh (hr)']==11)]

,#YY (#yr),MM (mo),DD (dy),hh (hr),mm (mn),WDIR (degT),WSPD (m/s),GST (m/s),WVHT (m),DPD (sec),APD (sec),MWD (degT),PRES (hPa),ATMP (degC),WTMP (degC),DEWP (degC),VIS (mi),TIDE (ft)
23952,2023,6,16,11,0,999,2.2,2.7,99.00,99.00,99.00,999,1005.9,999.0,18.5,999.0,99.0,99.0
23953,2023,6,16,11,10,999,2.1,2.7,0.64,7.14,4.95,158,1005.9,999.0,18.5,999.0,99.0,99.0
23954,2023,6,16,11,20,999,2.1,2.7,99.00,99.00,99.00,999,1005.9,999.0,18.5,999.0,99.0,99.0
23955,2023,6,16,11,30,999,3.1,4.5,99.00,99.00,99.00,999,1006.2,999.0,18.5,999.0,99.0,99.0
23956,2023,6,16,11,40,999,4.0,5.3,0.59,6.67,4.79,171,1006.1,999.0,18.5,999.0,99.0,99.0
23957,2023,6,16,11,50,999,4.3,5.3,99.00,99.00,99.00,999,1006.3,999.0,18.5,999.0,99.0,99.0


In [29]:
buoy_44065['WDIR (degT)'].unique()

array([194, 198, 207, 206, 203, 197, 195, 192, 191, 193, 190, 196, 201,
       202, 200, 214, 182, 181, 220, 241, 232, 223, 219, 229, 228, 235,
       224, 237, 247, 251, 252, 255, 256, 253, 254, 257, 258, 259, 261,
       249, 246, 226, 230, 242, 248, 243, 245, 250, 265, 266, 273, 280,
       283, 282, 279, 272, 262, 268, 274, 281, 284, 289, 292, 291, 297,
       299, 294, 288, 304, 317, 310, 270, 263, 260, 276, 239, 238, 234,
       236, 233, 269, 301, 221, 227, 231, 212, 225, 264, 267, 240, 216,
       218, 205, 199, 204, 209, 210, 208, 188, 148, 152, 140, 138, 146,
       172, 173, 165, 158, 161, 168, 166, 163, 164, 149, 151, 154, 156,
       159, 155, 143, 144, 153, 141, 127, 132, 134, 145, 169, 162, 167,
       176, 178, 177, 184, 186, 175, 189, 213, 217, 271, 179, 174, 185,
       187, 139, 133, 157, 160, 180, 171, 147, 150, 103,  72, 222, 275,
       306, 330, 346,  32,  27, 359, 355, 356, 333, 318, 315, 348, 357,
        43,  49,  30,  36,  50,  52,  53,  51,  42,  55,  59,  5